# Predicting Income With Census Data: Cleaning Data

Rafael G. Guerra | April 2022

There are a few things we must do before we begin any kind of analysis: (1) Give our data sets appropriate column names, (2) Ensure the features are coded properly (e.g. numbers should not be coded as strings), (3) Make the outcome variable into a binary 0 ( < 50,000) or 1 (50,000+), (4) Deal with missing variables, and (5) Perform One Hot Encoding for categorical variables

### Import Packages

In [1]:
import pandas as pd
import numpy as np

### Retrieve column names

In [2]:
metadata = open('census_income_metadata.txt', "r")
print(metadata.read(2641))

| This data was extracted from the census bureau database found at
| https://www.census.gov/data.html
| Donor: Terran Lane and Ronny Kohavi
|        Data Mining and Visualization
|        Silicon Graphics.
|        e-mail: terran@ecn.purdue.edu, ronnyk@sgi.com for questions.
|
| The data was split into train/test in approximately 2/3, 1/3
| proportions using MineSet's MIndUtil mineset-to-mlc.
|
| Prediction task is to determine the income level for the person
| represented by the record.  Incomes have been binned at the $50K
| level to present a binary classification problem, much like the
| original UCI/ADULT database.  The goal field of this data, however,
| was drawn from the "total person income" field rather than the
| "adjusted gross income" and may, therefore, behave differently than the
| original ADULT goal field.
|
| More information detailing the meaning of the attributes can be
| found in http://www.bls.census.gov/cps/cpsmain.htm
| To make use of the data descriptions at th

### Import Datasets and use Imported Colnames

In [126]:
colnames = ['AAGE','ACLSWKR','ADTIND','ADTOCC','AHGA','AHRSPAY','AHSCOL','AMARITL','AMJIND','AMJOCC',
            'ARACE','AREORGN','ASEX','AUNMEM','AUNTYPE','AWKSTAT','CAPGAIN','CAPLOSS','DIVVAL','FILESTAT',
            'GRINREG','GRINST','HHDFMX','HHDREL','MARSUPWT','MIGMTR1','MIGMTR3','MIGMTR4','MIGSAME',
            'MIGSUN','NOEMP','PARENT','PEFNTVTY','PEMNTVTY','PENATVTY','PRCITSHP','SEOTR','VETQVA',
            'VETYN','WKSWORK','YEAR','INCOME']

In [127]:
train_data = pd.read_csv('census_income_learn.csv', names=colnames)
test_data = pd.read_csv('census_income_test.csv', names=colnames)

### Let's have a glimpse at the datasets

In [128]:
train_data.head(3)

,AAGE,ACLSWKR,ADTIND,ADTOCC,AHGA,AHRSPAY,AHSCOL,AMARITL,AMJIND,AMJOCC,...,PEFNTVTY,PEMNTVTY,PENATVTY,PRCITSHP,SEOTR,VETQVA,VETYN,WKSWORK,YEAR,INCOME
0,73,Not in universe,0,0,High school graduate,0,Not in universe,Widowed,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,0,95,- 50000.
1,58,Self-employed-not incorporated,4,34,Some college but no degree,0,Not in universe,Divorced,Construction,Precision production craft & repair,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,94,- 50000.
2,18,Not in universe,0,0,10th grade,0,High school,Never married,Not in universe or children,Not in universe,...,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0,Not in universe,2,0,95,- 50000.


In [129]:
test_data.head(3)

,AAGE,ACLSWKR,ADTIND,ADTOCC,AHGA,AHRSPAY,AHSCOL,AMARITL,AMJIND,AMJOCC,...,PEFNTVTY,PEMNTVTY,PENATVTY,PRCITSHP,SEOTR,VETQVA,VETYN,WKSWORK,YEAR,INCOME
0,38,Private,6,36,1st 2nd 3rd or 4th grade,0,Not in universe,Married-civilian spouse present,Manufacturing-durable goods,Machine operators assmblrs & inspctrs,...,Mexico,Mexico,Mexico,Foreign born- Not a citizen of U S,0,Not in universe,2,12,95,- 50000.
1,44,Self-employed-not incorporated,37,12,Associates degree-occup /vocational,0,Not in universe,Married-civilian spouse present,Business and repair services,Professional specialty,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,26,95,- 50000.
2,2,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,95,- 50000.


### Verify variable types

If we see any class that is not of the type that they are supposed to be, we'll fix that. It is possible that whitespace is making an integer value be parsed as a string. We see that the only case this happens is with the INCOME variable. It actually doesn't matter that it's a string here because we will convert it to an int in just a second.

In [130]:
for variable in train_data:
    print(variable, train_data[variable][0], type(train_data[variable][0]))

AAGE 73 <class 'numpy.int64'>
ACLSWKR  Not in universe <class 'str'>
ADTIND 0 <class 'numpy.int64'>
ADTOCC 0 <class 'numpy.int64'>
AHGA  High school graduate <class 'str'>
AHRSPAY 0 <class 'numpy.int64'>
AHSCOL  Not in universe <class 'str'>
AMARITL  Widowed <class 'str'>
AMJIND  Not in universe or children <class 'str'>
AMJOCC  Not in universe <class 'str'>
ARACE  White <class 'str'>
AREORGN  All other <class 'str'>
ASEX  Female <class 'str'>
AUNMEM  Not in universe <class 'str'>
AUNTYPE  Not in universe <class 'str'>
AWKSTAT  Not in labor force <class 'str'>
CAPGAIN 0 <class 'numpy.int64'>
CAPLOSS 0 <class 'numpy.int64'>
DIVVAL 0 <class 'numpy.int64'>
FILESTAT  Nonfiler <class 'str'>
GRINREG  Not in universe <class 'str'>
GRINST  Not in universe <class 'str'>
HHDFMX  Other Rel 18+ ever marr not in subfamily <class 'str'>
HHDREL  Other relative of householder <class 'str'>
MARSUPWT 1700.09 <class 'numpy.float64'>
MIGMTR1  ? <class 'str'>
MIGMTR3  ? <class 'str'>
MIGMTR4  ? <class 'str

In [131]:
for variable in test_data:
    print(variable, test_data[variable][0], type(test_data[variable][0]))

AAGE 38 <class 'numpy.int64'>
ACLSWKR  Private <class 'str'>
ADTIND 6 <class 'numpy.int64'>
ADTOCC 36 <class 'numpy.int64'>
AHGA  1st 2nd 3rd or 4th grade <class 'str'>
AHRSPAY 0 <class 'numpy.int64'>
AHSCOL  Not in universe <class 'str'>
AMARITL  Married-civilian spouse present <class 'str'>
AMJIND  Manufacturing-durable goods <class 'str'>
AMJOCC  Machine operators assmblrs & inspctrs <class 'str'>
ARACE  White <class 'str'>
AREORGN  Mexican (Mexicano) <class 'str'>
ASEX  Female <class 'str'>
AUNMEM  Not in universe <class 'str'>
AUNTYPE  Not in universe <class 'str'>
AWKSTAT  Full-time schedules <class 'str'>
CAPGAIN 0 <class 'numpy.int64'>
CAPLOSS 0 <class 'numpy.int64'>
DIVVAL 0 <class 'numpy.int64'>
FILESTAT  Joint one under 65 & one 65+ <class 'str'>
GRINREG  Not in universe <class 'str'>
GRINST  Not in universe <class 'str'>
HHDFMX  Spouse of householder <class 'str'>
HHDREL  Spouse of householder <class 'str'>
MARSUPWT 1032.38 <class 'numpy.float64'>
MIGMTR1  ? <class 'str'>
M

### Recode Income

In [132]:
train_data['INCOME'] = np.where(train_data['INCOME'] == " - 50000.", 0, 1)
test_data['INCOME'] = np.where(test_data['INCOME'] == " - 50000.", 0, 1)

In [133]:
type(train_data['INCOME'][0])
type(test_data['INCOME'][0])

numpy.int64

### Deal with missing values

The first thing we need to do is to make Python realize it's missing data because right now it's just coded as a string '?'.

In [134]:
for variable in train_data:
    train_data[variable] = np.where(train_data[variable] == ' ?', np.nan, train_data[variable])

for variable in test_data:
    test_data[variable] = np.where(test_data[variable] == ' ?', np.nan, test_data[variable])

When we convert some variables that have a lot of '?', the class may become float instead of string. Important to verify which classes should be strings that are being coded as floats

In [138]:
type(train_data['MIGSUN'][0])

float

### How much missing data are we talking about?

Calculating the proportion of missing data in each column reveals the problem is mostly confined to four specific variables: MIGMTR1, MIGMTR3, MIGMTR4, and MIGSUN

In [139]:
percent_missing_train = train_data.isnull().sum() * 100 / len(train_data)
percent_missing_test = test_data.isnull().sum() * 100 / len(test_data)

In [140]:
percent_missing_train

AAGE         0.000000
ACLSWKR      0.000000
ADTIND       0.000000
ADTOCC       0.000000
AHGA         0.000000
AHRSPAY      0.000000
AHSCOL       0.000000
AMARITL      0.000000
AMJIND       0.000000
AMJOCC       0.000000
ARACE        0.000000
AREORGN      0.000000
ASEX         0.000000
AUNMEM       0.000000
AUNTYPE      0.000000
AWKSTAT      0.000000
CAPGAIN      0.000000
CAPLOSS      0.000000
DIVVAL       0.000000
FILESTAT     0.000000
GRINREG      0.000000
GRINST       0.354846
HHDFMX       0.000000
HHDREL       0.000000
MARSUPWT     0.000000
MIGMTR1     49.967172
MIGMTR3     49.967172
MIGMTR4     49.967172
MIGSAME      0.000000
MIGSUN      49.967172
NOEMP        0.000000
PARENT       0.000000
PEFNTVTY     3.364524
PEMNTVTY     3.066814
PENATVTY     1.700556
PRCITSHP     0.000000
SEOTR        0.000000
VETQVA       0.000000
VETYN        0.000000
WKSWORK      0.000000
YEAR         0.000000
INCOME       0.000000
dtype: float64

In [141]:
percent_missing_test

AAGE         0.000000
ACLSWKR      0.000000
ADTIND       0.000000
ADTOCC       0.000000
AHGA         0.000000
AHRSPAY      0.000000
AHSCOL       0.000000
AMARITL      0.000000
AMJIND       0.000000
AMJOCC       0.000000
ARACE        0.000000
AREORGN      0.000000
ASEX         0.000000
AUNMEM       0.000000
AUNTYPE      0.000000
AWKSTAT      0.000000
CAPGAIN      0.000000
CAPLOSS      0.000000
DIVVAL       0.000000
FILESTAT     0.000000
GRINREG      0.000000
GRINST       0.330787
HHDFMX       0.000000
HHDREL       0.000000
MARSUPWT     0.000000
MIGMTR1     50.065155
MIGMTR3     50.065155
MIGMTR4     50.065155
MIGSAME      0.000000
MIGSUN      50.065155
NOEMP        0.000000
PARENT       0.000000
PEFNTVTY     3.437180
PEMNTVTY     3.079329
PENATVTY     1.768208
PRCITSHP     0.000000
SEOTR        0.000000
VETQVA       0.000000
VETYN        0.000000
WKSWORK      0.000000
YEAR         0.000000
INCOME       0.000000
dtype: float64

### What to do?

There are a few alternatives: Delete rows with missing data, ignore the fact that the data is missing, impute values from other variables or the most frequent value in the column (i.e. mode), or delete the columns. Every solution has its drawbacks and its advantages. Since we will be having many features in this project and since there seems to be a systematic problem with four features, I decided to remove the four features from the analysis. 

In [143]:
# Remove MIGMTR1, MIGMTR3, MIGMT4, MIGSUN
train_data = train_data.drop(['MIGMTR1', 'MIGMTR3','MIGMTR4','MIGSUN'], axis=1)
test_data = test_data.drop(['MIGMTR1', 'MIGMTR3','MIGMTR4','MIGSUN'], axis=1)

### One Hot Encoding

One big thing we need to change here is that we can't just have the categorical data like it is, we need to transform it into something we can use for predictive models. Thus, we apply One Hot Encoding for categorical variables

In [145]:
categorical_variables_train = []
for variable in train_data:
    if(isinstance(train_data[variable][0], str)):
        categorical_variables_train.append(variable)
        
categorical_variables_test = []
for variable in test_data:
    if(isinstance(test_data[variable][0], str)):
        categorical_variables_test.append(variable)

In [149]:
categorical_variables_train

['ACLSWKR',
 'AHGA',
 'AHSCOL',
 'AMARITL',
 'AMJIND',
 'AMJOCC',
 'ARACE',
 'AREORGN',
 'ASEX',
 'AUNMEM',
 'AUNTYPE',
 'AWKSTAT',
 'FILESTAT',
 'GRINREG',
 'GRINST',
 'HHDFMX',
 'HHDREL',
 'MIGSAME',
 'PARENT',
 'PEFNTVTY',
 'PEMNTVTY',
 'PENATVTY',
 'PRCITSHP',
 'VETQVA']

In [151]:
cat_vars_train = pd.DataFrame()
for variable in categorical_variables_train:
    df = pd.get_dummies(train_data[variable],prefix=variable)
    cat_vars_train = pd.concat([cat_vars_train,df],axis=1)

cat_vars_test = pd.DataFrame()
for variable in categorical_variables_test:
    df = pd.get_dummies(test_data[variable],prefix=variable)
    cat_vars_test = pd.concat([cat_vars_test,df],axis=1)

In [153]:
cat_vars_train

,ACLSWKR_ Federal government,ACLSWKR_ Local government,ACLSWKR_ Never worked,ACLSWKR_ Not in universe,ACLSWKR_ Private,ACLSWKR_ Self-employed-incorporated,ACLSWKR_ Self-employed-not incorporated,ACLSWKR_ State government,ACLSWKR_ Without pay,AHGA_ 10th grade,...,PENATVTY_ Vietnam,PENATVTY_ Yugoslavia,PRCITSHP_ Foreign born- Not a citizen of U S,PRCITSHP_ Foreign born- U S citizen by naturalization,PRCITSHP_ Native- Born abroad of American Parent(s),PRCITSHP_ Native- Born in Puerto Rico or U S Outlying,PRCITSHP_ Native- Born in the United States,VETQVA_ No,VETQVA_ Not in universe,VETQVA_ Yes
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2,0,0,0,1,0,0,0,0,0,1,...,1,0,1,0,0,0,0,0,1,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199518,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
199519,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
199520,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
199521,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0


In [154]:
numerical_variables_train = []
for variable in train_data:
    if(isinstance(train_data[variable][0],str)==False):
        numerical_variables_train.append(variable)

numerical_variables_test = []
for variable in test_data:
    if(isinstance(test_data[variable][0],str)==False):
        numerical_variables_test.append(variable)

In [156]:
numerical_variables_train

['AAGE',
 'ADTIND',
 'ADTOCC',
 'AHRSPAY',
 'CAPGAIN',
 'CAPLOSS',
 'DIVVAL',
 'MARSUPWT',
 'NOEMP',
 'SEOTR',
 'VETYN',
 'WKSWORK',
 'YEAR',
 'INCOME']

In [163]:
num_vars_train = train_data[numerical_variables_train]
num_vars_test = test_data[numerical_variables_test]

### Combine Categorical (One Hot Encoded) and Numerical Features

In [164]:
train_clean_df = pd.concat([cat_vars_train,num_vars_train],axis=1)
test_clean_df = pd.concat([cat_vars_test,num_vars_test],axis=1)

In [165]:
train_clean_df

,ACLSWKR_ Federal government,ACLSWKR_ Local government,ACLSWKR_ Never worked,ACLSWKR_ Not in universe,ACLSWKR_ Private,ACLSWKR_ Self-employed-incorporated,ACLSWKR_ Self-employed-not incorporated,ACLSWKR_ State government,ACLSWKR_ Without pay,AHGA_ 10th grade,...,CAPGAIN,CAPLOSS,DIVVAL,MARSUPWT,NOEMP,SEOTR,VETYN,WKSWORK,YEAR,INCOME
0,0,0,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,1700.09,0.0,0.0,2.0,0.0,95.0,0.0
1,0,0,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,1053.55,1.0,0.0,2.0,52.0,94.0,0.0
2,0,0,0,1,0,0,0,0,0,1,...,0.0,0.0,0.0,991.95,0.0,0.0,2.0,0.0,95.0,0.0
3,0,0,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,1758.14,0.0,0.0,0.0,0.0,94.0,0.0
4,0,0,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,1069.16,0.0,0.0,0.0,0.0,94.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199518,0,0,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,955.27,0.0,0.0,2.0,0.0,95.0,0.0
199519,0,0,0,0,0,1,0,0,0,0,...,6418.0,0.0,9.0,687.19,1.0,0.0,2.0,52.0,94.0,0.0
199520,0,0,0,1,0,0,0,0,0,0,...,0.0,0.0,157.0,1923.03,6.0,0.0,2.0,52.0,95.0,0.0
199521,0,0,0,1,0,0,0,0,0,1,...,0.0,0.0,0.0,4664.87,0.0,0.0,2.0,0.0,95.0,0.0


### Generate Clean CSV

Let's generate two sets of data -- clean, and one hot encoded

In [166]:
train_clean_df.to_csv('train_clean.csv', index=False)
test_clean_df.to_csv('test_clean.csv', index=False)